<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Merging-main-files" data-toc-modified-id="Merging-main-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Merging main files</a></span></li><li><span><a href="#Loading-primers" data-toc-modified-id="Loading-primers-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Loading primers</a></span></li><li><span><a href="#DairyDB" data-toc-modified-id="DairyDB-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DairyDB</a></span></li></ul></div>

## Packages

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import re


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## DairyDB

In [33]:
from processing.hvr_extraction import get_all_hvr
from loading.main_sequence_loading import main_loading

In [64]:
df = main_loading('DairyDB')

In [71]:
df

,index,sequence,sequence_size,kingdom,phylum,class,order,family,genus,species,initial_db
299499,299499,GATGAACGCTGGCGGCGCGCCTAACACATGCAAGTCGAACGATTAA...,1464,BACTERIA,ACTINOBACTERIA,CORIOBACTERIIA,CORIOBACTERIALES,CORIOBACTERIACEAE,SLACKIA,SLACKIA_HELIOTRINIREDUCEN,DairyDB
299500,299500,AGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAATA...,1444,BACTERIA,PROTEOBACTERIA,GAMMAPROTEOBACTERIA,CELLVIBRIONALES,CELLVIBRIONACEAE,GILVIMARINUS,GILVIMARINUS_POLYSACCHAROLYTICU,DairyDB
299501,299501,AGAGTTTGATTATGGCTCAGGATGAACGCTAGCGGGAGGCTTAACA...,1477,BACTERIA,BACTEROIDETES,BACTEROIDIA,BACTEROIDALES,PROLIXIBACTERACEAE,DRACONIBACTERIUM,DRACONIBACTERIUM_SPECIE,DairyDB
299502,299502,GATGAACGCTAGCGGCAGGCCTAACACATGCAAGTCGAGGGGTAAC...,1468,BACTERIA,BACTEROIDETES,FLAVOBACTERIIA,FLAVOBACTERIALES,FLAVOBACTERIACEAE,YEOSUANA,YEOSUANA_AROMATIVORAN,DairyDB
299503,299503,AGAGTTTGATCCTGGCTCAGAGTGAACGCTGGCGGCGTGCTTAAGA...,1473,BACTERIA,PROTEOBACTERIA,DELTAPROTEOBACTERIA,SYNTROPHOBACTERALES,SYNTROPHACEAE,DESULFOMONILE,DESULFOMONILE_LIMIMARI,DairyDB
...,...,...,...,...,...,...,...,...,...,...,...
309883,309883,CCTGGCTCAGGATGAACGCTGGCGGCGTGCCAATACATGCAAGTCG...,1526,BACTERIA,FIRMICUTES,BACILLI,LACTOBACILLALES,LEUCONOSTOCACEAE,WEISSELLA,WEISSELLA_HELLENIC,DairyDB
309884,309884,AGAGTTTGATTNNGGCTCAGGATGAACGCTGGCGGCGTGCCTAATA...,1543,BACTERIA,FIRMICUTES,BACILLI,BACILLALES,STAPHYLOCOCCACEAE,STAPHYLOCOCCUS,STAPHYLOCOCCUS_HAEMOLYTICU,DairyDB
309885,309885,AGAGTTTGATNNTGGCTCAGGATGAACGCTGGCGGCGTGCCTAATA...,1543,BACTERIA,FIRMICUTES,BACILLI,BACILLALES,STAPHYLOCOCCACEAE,STAPHYLOCOCCUS,STAPHYLOCOCCUS_HOMINI,DairyDB
309886,309886,AACGCTGGCGGCGTGCCTAATACATGCAAGTCGAGCGAACAGACGA...,1469,BACTERIA,FIRMICUTES,BACILLI,BACILLALES,STAPHYLOCOCCACEAE,STAPHYLOCOCCUS,STAPHYLOCOCCUS_CAPRA,DairyDB


In [34]:
hvr_df = get_all_hvr(df)

In [83]:
hvr_df.loc[:, 'V1'].dropna()

0    GATTGAACGCTGGCGGCAGGCCTAATACATGCAAGTCGAGCGCGAA...
0    GGACGAACGCTGGCGGCATGCCTAATACATGCAAGTCGAACGAACT...
0    GGACGAACGCTGGCGGCGTGCTCCACACATGCAAGTCGAACGGAAA...
0    GGATGAACGCTGGCGGCGTGCCTAATACATGCAAGTCGAGCGAACA...
0    GGATGAACGCTGGCGGCGTGCCTAATACATGCAAGTCGAGCGAACG...
                           ...                        
0    GGATGAACGCTGGCGGCGTGCCTAATACATGCAAGTCGAGCGAACA...
0    GGATGAACACTGGCGGCGTGCCTAATACATGCAAGTCGAGCGAACA...
0    GGATGAACGCTGGCGGCGTGCCTAATACATGCAAGTCGAGCGAACA...
0    GGATGAACGCTGGCGGCGTGCCTAATACATGCAAGTCGAGCGAACA...
0    GGATGAACGCTGGCGGCGTGCCTAATACATGCAAGTCGAGCGAACA...
Name: V1, Length: 2372, dtype: object

In [43]:
from processing.hvr_statistics import main_stats

In [105]:
main_stats(df, hvr_df, sequence_origin='DairyDB', primers_origin='DairyDB')

-------------------------------------------------- GLOBAL INFORMATION --------------------------------------------------
Total number of observations: 10389
Average sequence size: 1446.56
Standard deviation: 67.53
Minimal size: 933
Maximal size: 2229
------------------------------------------------- TAROXOMY INFORMATION -------------------------------------------------
------------------------------------------------------- KINGDOM --------------------------------------------------------
Total number of unique kingdom: 2
Rank 1 in kingdom: BACTERIA with 10282 occurences
Rank 2 in kingdom: ARCHAEA with 107 occurences
0 missing values
-------------------------------------------------------- PHYLUM --------------------------------------------------------
Total number of unique phylum: 46
Rank 1 in phylum: FIRMICUTES with 3917 occurences
Rank 2 in phylum: PROTEOBACTERIA with 2310 occurences
Rank 3 in phylum: BACTEROIDETES with 1473 occurences
Rank 4 in phylum: ACTINOBACTERIA with 935 occur

Total number of remaining kingdom with at least one V67 found: 1 - 50.0% of total number of groups
Total number of remaining kingdom with at least one V78 found: 2 - 100.0% of total number of groups
-------------------------------------------------------- PHYLUM --------------------------------------------------------
Total number of unique phylum: 46
0 missing values - 0.0% of total
Total number of remaining phylum with at least one V1 found: 28 - 61.0% of total number of groups
Total number of remaining phylum with at least one V2 found: 30 - 65.0% of total number of groups
Total number of remaining phylum with at least one V3 found: 35 - 76.0% of total number of groups
Total number of remaining phylum with at least one V4 found: 44 - 96.0% of total number of groups
Total number of remaining phylum with at least one V5 found: 37 - 80.0% of total number of groups
Total number of remaining phylum with at least one V6 found: 39 - 85.0% of total number of groups
Total number of remaining

Total number of remaining species with at least one V4 found: 3852 - 96.0% of total number of groups
Total number of remaining species with at least one V5 found: 3380 - 84.0% of total number of groups
Total number of remaining species with at least one V6 found: 3453 - 86.0% of total number of groups
Total number of remaining species with at least one V7 found: 3706 - 92.0% of total number of groups
Total number of remaining species with at least one V9 found: 1907 - 47.0% of total number of groups
Total number of remaining species with at least one V12 found: 1455 - 36.0% of total number of groups
Total number of remaining species with at least one V23 found: 2473 - 61.0% of total number of groups
Total number of remaining species with at least one V34 found: 3742 - 93.0% of total number of groups
Total number of remaining species with at least one V45 found: 3424 - 85.0% of total number of groups
Total number of remaining species with at least one V56 found: 3758 - 93.0% of total nu

In [93]:
joined_df = pd.merge(df, hvr_df, on='index', how='left')


In [98]:
local_df = joined_df[['kingdom', 'V2']].dropna(axis=0)

In [100]:
depth='kingdom'

In [102]:
local_df[depth]

0        BACTERIA
1        BACTERIA
13       BACTERIA
15       BACTERIA
16       BACTERIA
           ...   
10384    BACTERIA
10385    BACTERIA
10386    BACTERIA
10387    BACTERIA
10388    BACTERIA
Name: kingdom, Length: 6092, dtype: object